This notebook turns raw data extracted from the data warehouse into bootstrapped data
ready for modeling purposes.

In [ ]:
import sys, os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import os

import pandas as pd
import numpy as np


In [ ]:
%load_ext autoreload
%autoreload 2

os.chdir('/home/adam/adam/causal_inference')

from causal_inference.data.load import *
from causal_inference.data.split import *
from causal_inference.data.impute import *
from causal_inference.data.bootstrap import *

In [ ]:
SEED_VALUE = 1234
OUTCOME = 'pf_ratio_12h_24h_manual_outcome'
TRAIN_SIZE = 0.8
BOOTSTRAP_SIZE = 0.95
RAW_DATA_PATH = ''
PROCESSED_DATA_PATH = ''

In [ ]:
np.random.seed(SEED_VALUE)

In [ ]:
os.chdir('/home/adam/adam/data/19012021/')
df = pd.read_csv('data_guerin_rct_fixed_prone.csv')
df.info(max_cols=200)

In [ ]:
df = prepare_csv_data(df=df, outcome_name=OUTCOME, threshold=0.7)
df.info()

In [ ]:
y, t, X = csv_data_to_np(df=df, outcome_name=OUTCOME, treatment_name='treated')

In [ ]:
print(y.shape)
print(t.shape)
print(X.shape)

In [ ]:
from causal_inference.data.split import train_test_split

y_train, t_train, X_train,\
    y_test, t_test, X_test = train_test_split(y, t, X, train_size=TRAIN_SIZE)

In [ ]:
from sklearn.model_selection import train_test_split

y_train, y_test, t_train, t_test,\
    X_train, X_test = train_test_split(y, t, X, train_size=TRAIN_SIZE, random_state=SEED_VALUE)

In [ ]:
print(y_train.shape)
print(t_train.shape)
print(X_train.shape)

print(y_test.shape)
print(t_test.shape)
print(X_test.shape)

In [ ]:
X_train, X_test = impute(X_train), impute(X_test)

In [ ]:
print(np.isnan(X_train).any())
print(np.isnan(X_test).any())

In [ ]:
train = bootstrap(y_train, t_train, X_train, bootstrap_size=BOOTSTRAP_SIZE, method='train')
test = bootstrap(y_test, t_test, X_test, bootstrap_size=BOOTSTRAP_SIZE, method='test')

In [ ]:
os.chdir('/home/adam/adam/cfrnet/data')

np.savez('bfpguerin_12_24.train.npz', yf=train[0], t=train[1], x=train[2])
np.savez('bfpguerin_12_24.test.npz', yf=test[0], t=test[1], x=test[2])